##### 小批量下载

In [8]:
import smallLib as helper
import redis as redis
import requests
import random
import numpy as np
import re
import os
import selenium.webdriver as webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.wait import WebDriverWait #等待页面加载某些元素
from selenium.webdriver.support import expected_conditions as EC

In [9]:
import json
from tempfile import tempdir

pool = redis.ConnectionPool(host='127.0.0.1',port=6379,db =1,password='123456',decode_responses=True)   
r = redis.Redis(connection_pool=pool)
temp_table = 'all_user'
def restore_table(pool,origin,temp): 
    r = redis.Redis(connection_pool=pool)
    table = r.hgetall(origin)
    temp_table = temp
    r.hmset(temp_table, table)

#设置容忍度，若一个用户主页中下载成功的视频不足总视频量的tolerence，则重新开始下载该用户主页

def check_empty_dir_and_update_redis(farther_dir,table,tolerence):
    need_retry={}
    for listx in os.listdir(farther_dir):
        num = len(os.listdir(farther_dir+os.sep+listx))
        ls  = r.hget(table,listx)
        if num > len(ls.split(','))*tolerence:
            #print(listx)
            helper.delete_user(r,table,listx)

In [10]:
from time import time

pair=[1,1]
user= [1,1]
video_info = {};
while r.hlen(temp_table)>0:
    name = r.get('curr')
    #可从中断处返回

    if name == None:
        user= helper.get_user(r,temp_table)
        name = user[0]
        r.set('curr',name)
        urls = re.sub('[\[\] \']','',user[1]).split(',')
        for url in urls:
            r.hset(name,url,0)

    while r.hlen(name) > 0:
        url = helper.redis_hpop(r,name)  
        try :
            fetch_info =   helper.down_load_video_from_user_homepage(redis.ConnectionPool(host='127.0.0.1',port=6379,db =2,password='123456',decode_responses=True)  ,name,url)
            video_info[url] = fetch_info
            r.hdel(name,url)
            time.sleep(random.uniform(0.1,1.1))
        except Exception:
            continue
        
    r.delete('curr',name)
    helper.delete_user(r,temp_table,name)

KeyboardInterrupt: 

In [ ]:
#check_empty_dir_and_update_redis("./video")


C:\Users\Dr.Seydlitz\AppData\Local\Temp\ipykernel_110668\495543122.py:14: DeprecationWarning: Redis.hmset() is deprecated. Use Redis.hset() instead.
  r.hmset(temp_table, user)
